## Daily Candles


looking at the candle formations of the daily candle to take trades, this will ideally act as an entry signal for long term holds

In [91]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
from talib import CDLENGULFING, CDLMORNINGSTAR, CDLSHOOTINGSTAR 
from backtesting import Backtest, Strategy
mt5.initialize()
    
def get_data(symbol, timeframe, start_date, end_time):
    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_time)
    df = pd.DataFrame(rates)
    df = df[['time', 'open', 'high', 'low', 'close']]
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    return df

def get_engulfing(symbol):
    symbol['engulfing'] = CDLENGULFING(symbol.Open, symbol.High, symbol.Low, symbol.Close)
    return symbol

def get_morningstar(symbol):
    symbol['morningstar'] = CDLMORNINGSTAR(symbol.Open, symbol.High, symbol.Low, symbol.Close)
    return symbol 

def get_shootingstar(symbol):
    symbol['shootingstar'] = CDLSHOOTINGSTAR(symbol.Open, symbol.High, symbol.Low, symbol.Close)
    return symbol  

def SIGNAL():
    return eurusd.engulfing

class Candlesticks(Strategy):
    initsize = 500.0
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        
    def next(self):
        super().next()
        if self.signal1 == 100:
            sl1 = self.data.Close*0.99
            tp1 = self.data.Close*1.02
            self.buy(sl = sl1, tp = tp1, size=self.mysize)
        if self.signal1 == -100:
            sl1 = self.data.Close*1.01
            tp1 = self.data.Close*0.98
            self.sell(sl = sl1, tp = tp1, size=self.mysize)    
            
if __name__ == '__main__':
    # Settings
    symbol = 'EURUSD'
    timeframe = mt5.TIMEFRAME_D1
    start_date = datetime(2024, 1, 1)
    end_time = datetime.now()
    # Get data and analysing
    eurusd = get_data(symbol, timeframe, start_date, end_time)
    eurusd.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    eurusd = get_engulfing(eurusd)
    eurusd = get_morningstar(eurusd)
    eurusd = get_shootingstar(eurusd)

    # Conducting backtest
    bt = Backtest(eurusd, Candlesticks, cash=10000, exclusive_orders=True, margin= 1/100)
    # Outputs
    output = bt.run()
    plot = bt.plot()
    print(output)


Start                     2024-01-02 00:00:00
End                       2024-05-21 00:00:00
Duration                    140 days 00:00:00
Exposure Time [%]                   52.252252
Equity Final [$]                    9985.6254
Equity Peak [$]                       10001.1
Return [%]                          -0.143746
Buy & Hold Return [%]               -0.810608
Return (Ann.) [%]                   -0.326044
Volatility (Ann.) [%]                0.185059
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.198074
Avg. Drawdown [%]                   -0.071991
Max. Drawdown Duration      131 days 00:00:00
Avg. Drawdown Duration       46 days 00:00:00
# Trades                                   14
Win Rate [%]                        42.857143
Best Trade [%]                       0.817615
Worst Trade [%]                          -1.0
Avg. Trade [%]                    